In [1]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.endpoints import leaguestandingsv3
from datetime import datetime, timezone, timedelta
from dateutil import parser
from nba_api.live.nba.endpoints import scoreboard
from nba_api.stats.endpoints import boxscoresummaryv3
import sys
from pathlib import Path
import pandas as pd


# Get the parent directory (Datos folder)
current_dir = Path().resolve()  # libretas folder
datos_dir = current_dir.parent  # Datos folder

# Add Datos to path so we can import from Functions
sys.path.insert(0, str(datos_dir))



pd.set_option("display.max_columns", None)


In [2]:

from helpfuncs import get_current_season
import numpy as np
from typing import List, Dict, Tuple, Union, Optional

In [3]:
class Games():
    def __init__(self):
        self.current_season = get_current_season()

    def check_valid_season(self, season: str) -> str:
        """Check if the provided season is valid and return formatted season."""
        if not season:
            season = self.current_season
        elif not season.isdigit():
            raise ValueError(f"Invalid season format: {season}. Expected a year in 'YYYY' format.")
        elif int(season) < 1980 or int(season) > datetime.now().year:
            raise ValueError(f"Invalid season year: {season}. Year must be between 1980 and {datetime.now().year}.")
        else:
            season = f"{season}-{str(int(season)+1)[-2:]}"
        
        return season

    def get_team_game_log(self, team_id: int, season: str) -> pd.DataFrame:
        game_log = teamgamelog.TeamGameLog(team_id=team_id, season=season)
        game_log = game_log.get_data_frames()[0]
        return game_log

    def get_todays_games(self) -> None:
        f = "{gameId}: {awayTeam} @ {homeTeam} : {gameTimeLTZ}" 
        board = scoreboard.ScoreBoard()
        print("ScoreBoardDate: " + board.score_board_date)
        games = board.games.get_dict()
        for game in games:
            gameTimeLTZ = parser.parse(game["gameTimeUTC"]).replace(tzinfo=timezone.utc).astimezone(tz=None)
            print(f.format(gameId=game['gameId'], awayTeam=game['awayTeam']['teamName'], homeTeam=game['homeTeam']['teamName'], gameTimeLTZ=gameTimeLTZ))
        
        return games
    
    def get_game_by_id(self, game_id: str) -> Dict:
        """Get game details by game ID"""
        try:
            
            game_summary = boxscoresummaryv3.BoxScoreSummaryV3(game_id=game_id)
            
            # Get all dataframes
            game_summary_data = game_summary.get_data_frames()[7]
            line_score = game_summary.get_data_frames()[1]
            
            return game_summary_data
            
        except Exception as e:
            print(f"Error retrieving game {game_id}: {e}")
            raise e

    def get_current_standings(self, season: str = None, conference:str = 'Overall') -> pd.DataFrame:
        """Get current NBA standings for all teams"""
        conference_types = ['Overall','West','East']
        try:
            if conference in conference_types:
                season = self.check_valid_season(season)
                
                standings = leaguestandingsv3.LeagueStandingsV3(
                    league_id='00',  # NBA
                    season=season,
                    season_type='Regular Season'
                )
                
                standings_df = standings.get_data_frames()[0]
                
                # Select relevant columns
                columns_to_keep = [
                    'TeamID', 'TeamCity', 'TeamName', 'Conference','WINS', 'LOSSES', 'PlayoffRank', 
                    'Record', "HOME","ROAD", "L10"
                ]
                
                standings_df = standings_df[columns_to_keep]

                if conference != 'Overall':
                    return standings_df[standings_df['Conference'] == conference].reset_index(drop=True)
                else:
                    return standings_df
            else:
                raise ValueError(f"Invalid conference type: {conference}. Expected one of {conference_types}.")    
        except Exception as e:
            print(f"Error retrieving standings for season {season}: {e}")
            raise e


In [6]:
#Games().get_game_by_id(game_id='0022500278')
#Games().get_current_standings(season='2025', conference='Overall')
games = Games().get_todays_games()

ScoreBoardDate: 2025-12-18
0022500371: Hawks @ Hornets : 2025-12-18 17:00:00-07:00
0022500372: Knicks @ Pacers : 2025-12-18 17:00:00-07:00
0022500373: Heat @ Nets : 2025-12-18 17:30:00-07:00
0022500369: Clippers @ Thunder : 2025-12-18 18:00:00-07:00
0022500370: Wizards @ Spurs : 2025-12-18 18:00:00-07:00
0022500374: Raptors @ Bucks : 2025-12-18 18:00:00-07:00
0022500375: Rockets @ Pelicans : 2025-12-18 18:00:00-07:00
0022500376: Pistons @ Mavericks : 2025-12-18 18:30:00-07:00
0022500377: Magic @ Nuggets : 2025-12-18 19:00:00-07:00
0022500378: Warriors @ Suns : 2025-12-18 19:00:00-07:00
0022500379: Lakers @ Jazz : 2025-12-18 19:00:00-07:00
0022500380: Kings @ Trail Blazers : 2025-12-18 20:00:00-07:00


In [7]:
games

[{'gameId': '0022500371',
  'gameCode': '20251218/ATLCHA',
  'gameStatus': 1,
  'gameStatusText': '7:00 pm ET',
  'period': 0,
  'gameClock': '',
  'gameTimeUTC': '2025-12-19T00:00:00Z',
  'gameEt': '2025-12-18T19:00:00Z',
  'regulationPeriods': 4,
  'ifNecessary': False,
  'seriesGameNumber': '',
  'gameLabel': '',
  'gameSubLabel': '',
  'seriesText': '',
  'seriesConference': '',
  'poRoundDesc': '',
  'gameSubtype': '',
  'isNeutral': False,
  'homeTeam': {'teamId': 1610612766,
   'teamName': 'Hornets',
   'teamCity': 'Charlotte',
   'teamTricode': 'CHA',
   'wins': 8,
   'losses': 18,
   'score': 0,
   'seed': None,
   'inBonus': None,
   'timeoutsRemaining': 0,
   'periods': [{'period': 1, 'periodType': 'REGULAR', 'score': 0},
    {'period': 2, 'periodType': 'REGULAR', 'score': 0},
    {'period': 3, 'periodType': 'REGULAR', 'score': 0},
    {'period': 4, 'periodType': 'REGULAR', 'score': 0}]},
  'awayTeam': {'teamId': 1610612737,
   'teamName': 'Hawks',
   'teamCity': 'Atlanta',


In [21]:
print(games)

[{'gameId': '0062500001', 'gameCode': '20251216/SASNYK', 'gameStatus': 3, 'gameStatusText': 'Final', 'period': 4, 'gameClock': '', 'gameTimeUTC': '2025-12-17T01:30:00Z', 'gameEt': '2025-12-16T20:30:00Z', 'regulationPeriods': 4, 'ifNecessary': False, 'seriesGameNumber': '', 'gameLabel': 'Emirates NBA Cup', 'gameSubLabel': 'Championship', 'seriesText': 'Neutral Site', 'seriesConference': '', 'poRoundDesc': '', 'gameSubtype': 'in-season-knockout', 'isNeutral': True, 'homeTeam': {'teamId': 1610612752, 'teamName': 'Knicks', 'teamCity': 'New York', 'teamTricode': 'NYK', 'wins': 1, 'losses': 0, 'score': 124, 'seed': 3, 'inBonus': None, 'timeoutsRemaining': 0, 'periods': [{'period': 1, 'periodType': 'REGULAR', 'score': 28}, {'period': 2, 'periodType': 'REGULAR', 'score': 31}, {'period': 3, 'periodType': 'REGULAR', 'score': 30}, {'period': 4, 'periodType': 'REGULAR', 'score': 35}]}, 'awayTeam': {'teamId': 1610612759, 'teamName': 'Spurs', 'teamCity': 'San Antonio', 'teamTricode': 'SAS', 'wins': 

In [ ]:
f = "{gameId}: {awayTeam} @ {homeTeam} : {gameTimeLTZ}" 
board = scoreboard.ScoreBoard()
print("ScoreBoardDate: " + board.score_board_date)
games = board.games.get_dict()
for game in games:
    gameTimeLTZ = parser.parse(game["gameTimeUTC"]).replace(tzinfo=timezone.utc).astimezone(tz=None)
    print(f.format(gameId=game['gameId'], awayTeam=game['awayTeam']['teamName'], homeTeam=game['homeTeam']['teamName'], gameTimeLTZ=gameTimeLTZ))




ScoreBoardDate: 2025-12-16
0062500001: Spurs @ Knicks : 2025-12-16 18:30:00-07:00


In [17]:
games

[{'gameId': '0062500001',
  'gameCode': '20251216/SASNYK',
  'gameStatus': 3,
  'gameStatusText': 'Final',
  'period': 4,
  'gameClock': '',
  'gameTimeUTC': '2025-12-17T01:30:00Z',
  'gameEt': '2025-12-16T20:30:00Z',
  'regulationPeriods': 4,
  'ifNecessary': False,
  'seriesGameNumber': '',
  'gameLabel': 'Emirates NBA Cup',
  'gameSubLabel': 'Championship',
  'seriesText': 'Neutral Site',
  'seriesConference': '',
  'poRoundDesc': '',
  'gameSubtype': 'in-season-knockout',
  'isNeutral': True,
  'homeTeam': {'teamId': 1610612752,
   'teamName': 'Knicks',
   'teamCity': 'New York',
   'teamTricode': 'NYK',
   'wins': 1,
   'losses': 0,
   'score': 124,
   'seed': 3,
   'inBonus': None,
   'timeoutsRemaining': 0,
   'periods': [{'period': 1, 'periodType': 'REGULAR', 'score': 28},
    {'period': 2, 'periodType': 'REGULAR', 'score': 31},
    {'period': 3, 'periodType': 'REGULAR', 'score': 30},
    {'period': 4, 'periodType': 'REGULAR', 'score': 35}]},
  'awayTeam': {'teamId': 1610612759